# A collection of issues that I encountered working with CMIP6 models. Ill try to address them in PRs at some point.

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import intake
from cmip6_preprocessing.preprocessing import combined_preprocessing
from dask.diagnostics import ProgressBar
import warnings

%load_ext autoreload
%autoreload 2
%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 6
%config InlineBackend.figure_format = 'retina'
xr.set_options(cmap_sequential='cividis', display_style='html')

In [8]:
col = intake.open_esm_datastore("https://storage.googleapis.com/cmip6/pangeo-cmip6.json")

#exper = 'historical'
exper = 'piSST-pdSIC'

query = dict(variable_id=['tas'],
             experiment_id = exper,
             table_id=['Amon'], 
             source_id=['NorESM2-LM'],
             grid_label=['gn'],
            )
cat = col.search(**query)
display(cat.df)

/usr/local/python/anaconda3/envs/pangeo-forge/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,PAMIP,NCC,NorESM2-LM,piSST-pdSIC,r100i1p1f1,Amon,tas,gn,gs://cmip6/PAMIP/NCC/NorESM2-LM/piSST-pdSIC/r1...,NaN,20190920
1,PAMIP,NCC,NorESM2-LM,piSST-pdSIC,r10i1p1f1,Amon,tas,gn,gs://cmip6/PAMIP/NCC/NorESM2-LM/piSST-pdSIC/r1...,NaN,20190920
2,PAMIP,NCC,NorESM2-LM,piSST-pdSIC,r11i1p1f1,Amon,tas,gn,gs://cmip6/PAMIP/NCC/NorESM2-LM/piSST-pdSIC/r1...,NaN,20190920
3,PAMIP,NCC,NorESM2-LM,piSST-pdSIC,r12i1p1f1,Amon,tas,gn,gs://cmip6/PAMIP/NCC/NorESM2-LM/piSST-pdSIC/r1...,NaN,20190920
4,PAMIP,NCC,NorESM2-LM,piSST-pdSIC,r14i1p1f1,Amon,tas,gn,gs://cmip6/PAMIP/NCC/NorESM2-LM/piSST-pdSIC/r1...,NaN,20190920
...,...,...,...,...,...,...,...,...,...,...,...
83,PAMIP,NCC,NorESM2-LM,piSST-pdSIC,r96i1p1f1,Amon,tas,gn,gs://cmip6/PAMIP/NCC/NorESM2-LM/piSST-pdSIC/r9...,NaN,20190920
84,PAMIP,NCC,NorESM2-LM,piSST-pdSIC,r97i1p1f1,Amon,tas,gn,gs://cmip6/PAMIP/NCC/NorESM2-LM/piSST-pdSIC/r9...,NaN,20190920
85,PAMIP,NCC,NorESM2-LM,piSST-pdSIC,r98i1p1f1,Amon,tas,gn,gs://cmip6/PAMIP/NCC/NorESM2-LM/piSST-pdSIC/r9...,NaN,20190920
86,PAMIP,NCC,NorESM2-LM,piSST-pdSIC,r99i1p1f1,Amon,tas,gn,gs://cmip6/PAMIP/NCC/NorESM2-LM/piSST-pdSIC/r9...,NaN,20190920


In [9]:
data_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times':False}, aggregate=True)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [7]:
#ds.lev_bnds.isel(time=0).drop_vars(['time'])
data_dict['PAMIP.NCC.NorESM2-LM.piSST-pdSIC.Amon.gn']

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 96, lon: 144, member_id: 88, time: 12)
Coordinates:
    height     float64 ...
  * lat        (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(96, 2), meta=np.ndarray>
  * lon        (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(144, 2), meta=np.ndarray>
  * time       (time) int64 0 732 1476 2208 2940 ... 5148 5856 6564 7296 8028
    time_bnds  (time, bnds) float64 dask.array<chunksize=(12, 2), meta=np.ndarray>
  * member_id  (member_id) <U10 'r100i1p1f1' 'r10i1p1f1' ... 'r9i1p1f1'
Dimensions without coordinates: bnds
Data variables:
    tas        (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 6, 96, 144), meta=np.ndarray>
Attributes:
    sub_experiment_id:         none
    mip_era:                   CMIP6
    parent_activity_id:        CMIP
    institution:               NorESM Climate modeling Consortium consisting ...
    branch_time:               0.0
    forcing_index:             1
    parent_variant_label:      r1i1p1f1
    grid_label:                gn
    variable_id:               tas
    activity_id:               PAMIP
    title:                     NorESM2-LM output prepared for CMIP6
    parent_sub_experiment_id:  none
    branch_time_in_parent:     9215.0
    Conventions:               CF-1.7 CMIP-6.2
    realm:                     atmos
    model_id:                  NorESM2-LM
    sub_experiment:            none
    experiment_id:             piSST-pdSIC
    table_info:                Creation Date:(24 July 2019) MD5:0bb394a356ef9...
    contact:                   Please send any requests or bug reports to nor...
    institution_id:            NCC
    history:                   2019-10-09T06:39:19Z ; CMOR rewrote data to be...
    parent_time_units:         days since 1975-01-01 00:00:00
    cmor_version:              3.5.0
    branch_method:             Hybrid-restart from year 2000-04-01 of amip
    source:                    NorESM2-LM (2017): \naerosol: OsloAero\natmos:...
    parent_mip_era:            CMIP6
    source_type:               AGCM AER
    frequency:                 mon
    license:                   CMIP6 model data produced by NCC is licensed u...
    status:                    2020-11-06;created; by gcs.cmip6.ldeo@gmail.com
    run_variant:               N/A
    tracking_id:               hdl:21.14100/7941f69d-783d-462b-abb9-f531027cd...
    grid:                      finite-volume grid with 1.9x2.5 degree lat/lon...
    parent_source_id:          NorESM2-LM
    branch_time_in_child:      0.0
    external_variables:        areacella
    initialization_index:      1
    data_specs_version:        01.00.31
    intake_esm_varname:        ['tas']
    product:                   model-output
    experiment:                Atmosphere time slice with pre-industrial SST ...
    source_id:                 NorESM2-LM
    physics_index:             1
    table_id:                  Amon
    nominal_resolution:        250 km
    parent_experiment_id:      amip
    intake_esm_dataset_key:    PAMIP.NCC.NorESM2-LM.piSST-pdSIC.Amon.gn